In [49]:
import pandas as pd
import numpy as np
import time
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
pd.set_option('display.max_row', None)

In [56]:
standings_2025 = pd.read_csv('standings_2025_before_all_star.csv')
standings_2006_2024 = pd.read_csv('end_season_standings_2006_2024.csv')
standings_2006_2024_before_allstar = pd.read_csv('standings_2006_2024_before_all_star.csv')


In [57]:
#Adding more Attributes about each Team (home wins/losses, remaining games, Strength of Schedule Remaining etc)

standings_2025["home_wins"] = standings_2025['HOME'].str.rstrip('-').str[:2].astype(int)
standings_2025['home_losses'] = standings_2025['HOME'].str.rstrip('-').str[3:].astype(int)
standings_2025["away_wins"] = standings_2025['AWAY'].str.rstrip('-').str[:2].astype(int)
standings_2025['away_losses'] = standings_2025['AWAY'].str.rstrip('-').str[3:].astype(int)


standings_2025["home_games_remaining"] = 81 - standings_2025["home_wins"] - standings_2025["home_losses"]
standings_2025["away_games_remaining"] = 81 - standings_2025["away_wins"] - standings_2025["away_losses"]


standings_2025['SOS_remaining'] = [
    0.489,  # Detroit Tigers
    0.481,  # Los Angeles Dodgers
    0.482,  # Chicago Cubs
    0.479,  # Houston Astros
    0.511,  # Milwaukee Brewers
    0.497,  # Philadelphia Phillies
    0.503,  # Toronto Blue Jays
    0.511,  # New York Mets
    0.498,  # New York Yankees
    0.485,  # San Diego Padres
    0.509,  # Boston Red Sox
    0.488,  # San Francisco Giants
    0.487,  # Seattle Mariners
    0.500,  # St. Louis Cardinals
    0.521,  # Cincinnati Reds
    0.501,  # Tampa Bay Rays
    0.513,  # Texas Rangers
    0.504,  # Los Angeles Angels
    0.502,  # Minnesota Twins
    0.494,  # Arizona Diamondbacks
    0.498,  # Kansas City Royals
    0.480,  # Cleveland Guardians
    0.508,  # Miami Marlins
    0.515,  # Baltimore Orioles
    0.507,  # Atlanta Braves
    0.507,  # Athletics Athletics
    0.496,  # Pittsburgh Pirates
    0.503,  # Washington Nationals
    0.513,  # Chicago White Sox
    0.516   # Colorado Rockies
]


standings_2025['SOS_final'] = (((standings_2025['W']+ standings_2025['L']) * standings_2025['SOS']) + ((162-(standings_2025['W']+ standings_2025['L'])) * standings_2025['SOS_remaining']))/162
standings_2025['Pythag_Wins'] = (standings_2025['RS'] ** 1.83) / (standings_2025['RS'] ** 1.83 + standings_2025['RA'] ** 1.83) * (standings_2025['W']+ standings_2025['L'])
standings_2025['Win_Diff'] = standings_2025['W'] - standings_2025['Pythag_Wins']

standings_2025.columns = standings_2025.columns.map(lambda col: col + "_mid")


In [58]:
standings_2025 = standings_2025.rename(columns = {'SOS_final_mid': "SOS_final", 'home_games_remaining_mid': "home_games_remaining","away_games_remaining_mid": "away_games_remaining" })

In [59]:
#Adding the same attributes 

historical = pd.merge(standings_2006_2024_before_allstar,standings_2006_2024, on = ['TEAM', "Year"], how = 'inner', suffixes=('_mid', '_final'))
historical["Final_Wins"] = historical["W_final"]

historical["home_wins_mid"] = historical['HOME_mid'].str.rstrip('-').str[:2].astype(int)
historical['home_losses_mid'] = historical['HOME_mid'].str.rstrip('-').str[3:].astype(int)
historical['away_losses_mid'] = historical['AWAY_mid'].str.rstrip('-').str[3:].astype(int)
historical['away_wins_mid'] = historical['AWAY_mid'].apply(
    lambda x: int(x.split('-')[0]) if '-' in x else int(x))
historical["home_games_remaining"] = 81 - historical["home_wins_mid"] - historical["home_losses_mid"]
historical["away_games_remaining"] = 81 - historical["away_wins_mid"] - historical["away_losses_mid"]

In [61]:
#adding the features for the ML algorithm and running training and testing using Time Series Split 
#Looking at the Mean Absolute Error for Each Test Set

features = ['W_mid','RS_mid', 'RA_mid', 'DIFF_mid', 'SOS_final', 'Home_Win_PCT_mid',
            'Away_Win_PCT_mid', 'home_games_remaining', 'away_games_remaining'
            ]
target = 'Final_Wins'
tscv = TimeSeriesSplit(n_splits=5)

# Optimized XGBoost model
model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)


train_data = historical
train_data = train_data.sort_values('Year')

tscv = TimeSeriesSplit(n_splits=5)


for train_index, test_index in tscv.split(train_data):
    train_years = np.unique(train_data.iloc[train_index]['Year'])
    test_years = np.unique(train_data.iloc[test_index]['Year'])
    print(f"Training Years: {min(train_years)}-{max(train_years)} ({len(train_years)} seasons)")
    print(f"Testing Years: {min(test_years)}-{max(test_years)} ({len(test_years)} seasons)")
    X_train, X_test = train_data.iloc[train_index][features], train_data.iloc[test_index][features]
    y_train, y_test = train_data.iloc[train_index][target], train_data.iloc[test_index][target]
    
    model.fit(X_train, y_train)
    val_preds = model.predict(X_test)
    val_mae = mean_absolute_error(y_test, val_preds)
    print(f"Fold MAE: {val_mae:.2f}")

Training Years: 2006-2008 (3 seasons)
Testing Years: 2009-2011 (3 seasons)
Fold MAE: 5.17
Training Years: 2006-2011 (6 seasons)
Testing Years: 2012-2014 (3 seasons)
Fold MAE: 4.83
Training Years: 2006-2014 (9 seasons)
Testing Years: 2015-2017 (3 seasons)
Fold MAE: 4.64
Training Years: 2006-2017 (12 seasons)
Testing Years: 2018-2021 (3 seasons)
Fold MAE: 5.88
Training Years: 2006-2021 (15 seasons)
Testing Years: 2022-2024 (3 seasons)
Fold MAE: 4.69


In [62]:
#Projected Standings based on my Model

X_2025 = standings_2025[features]
standings_2025['Projected_Final_Wins'] = model.predict(X_2025)
standings_2025["Win_Range"] = [f"{round(wins-5), round(wins+5)}" for wins in standings_2025['Projected_Final_Wins']]
projected_standings = standings_2025[['TEAM_mid', 'Projected_Final_Wins', "Win_Range"]]
projected_standings = projected_standings.rename(columns = {'TEAM_mid': "Team"}).sort_values('Projected_Final_Wins', ascending = False)
projected_standings['Projected_Final_Wins'] = projected_standings['Projected_Final_Wins'].round()
projected_standings

,Team,Projected_Final_Wins,Win_Range
2,Chicago Cubs,100.0,"(95, 105)"
0,Detroit Tigers,96.0,"(91, 101)"
4,Milwaukee Brewers,96.0,"(91, 101)"
5,Philadelphia Phillies,95.0,"(90, 100)"
3,Houston Astros,93.0,"(88, 98)"
1,Los Angeles Dodgers,93.0,"(88, 98)"
7,New York Mets,93.0,"(88, 98)"
10,Boston Red Sox,93.0,"(88, 98)"
8,New York Yankees,92.0,"(87, 97)"
6,Toronto Blue Jays,90.0,"(85, 95)"


In [63]:
#Fangraphs Projected Final Standings

data = {
    "Team": [
        "Los Angeles Angels", "Houston Astros", "Athletics Athletics", "Toronto Blue Jays",
        "Atlanta Braves", "Milwaukee Brewers", "St. Louis Cardinals", "Chicago Cubs",
        "Arizona Diamondbacks", "Los Angeles Dodgers", "San Francisco Giants",
        "Cleveland Guardians", "Seattle Mariners", "Miami Marlins", "New York Mets",
        "Washington Nationals", "Baltimore Orioles", "San Diego Padres", "Philadelphia Phillies",
        "Pittsburgh Pirates", "Texas Rangers", "Tampa Bay Rays", "Boston Red Sox",
        "Cincinnati Reds", "Colorado Rockies", "Kansas City Royals", "Detroit Tigers",
        "Minnesota Twins", "Chicago White Sox", "New York Yankees"
    ],
    "Fangraphs_Win_Projections": [
        76, 91, 70, 91, 
        77, 92, 82, 91,
        80, 94, 84, 80,
        87, 76, 91, 68,
        75, 85, 91, 70,
        83, 83, 84, 81,
        50, 79, 90, 80,
        61, 89
    ]
}
fangraphs_mapping_full = pd.DataFrame(data)

In [64]:
#Comparing Fangraphs and My Models Final Standings

myprojections_vs_fangraphs = pd.merge(projected_standings, fangraphs_mapping_full, on = 'Team', how = 'left')
myprojections_vs_fangraphs['Difference'] = myprojections_vs_fangraphs['Projected_Final_Wins'] - myprojections_vs_fangraphs['Fangraphs_Win_Projections'] 
myprojections_vs_fangraphs

,Team,Projected_Final_Wins,Win_Range,Fangraphs_Win_Projections,Difference
0,Chicago Cubs,100.0,"(95, 105)",91,9.0
1,Detroit Tigers,96.0,"(91, 101)",90,6.0
2,Milwaukee Brewers,96.0,"(91, 101)",92,4.0
3,Philadelphia Phillies,95.0,"(90, 100)",91,4.0
4,Houston Astros,93.0,"(88, 98)",91,2.0
5,Los Angeles Dodgers,93.0,"(88, 98)",94,-1.0
6,New York Mets,93.0,"(88, 98)",91,2.0
7,Boston Red Sox,93.0,"(88, 98)",84,9.0
8,New York Yankees,92.0,"(87, 97)",89,3.0
9,Toronto Blue Jays,90.0,"(85, 95)",91,-1.0
